
# WEEK 13

2024/07/01 - 2024/07/07


## 深度学习 C4_W2

### 1. 使用 Keras 实现 ResNets 模块

- **ResNets 模块**：
  - 主要特点是残差块（Residual Block），通过短路连接（skip connections）来解决梯度消失和梯度爆炸问题。
  - 残差块包含两个或多个卷积层，并在卷积层之间添加一个直接的跳跃连接，这种结构允许梯度直接从输出层传递到输入层，从而减少了梯度消失的问题。

- **代码示例**：

```python
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Add, ReLU
from tensorflow.keras.models import Model

def resnet_block(input_tensor, filters, kernel_size=3, stride=1):
    # First convolutional layer
    x = Conv2D(filters, kernel_size, strides=stride, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Second convolutional layer
    x = Conv2D(filters, kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    
    # Add skip connection
    x = Add()([x, input_tensor])
    x = ReLU()(x)
    return x

input_tensor = Input(shape=(64, 64, 3))
output_tensor = resnet_block(input_tensor, filters=64)
model = Model(inputs=input_tensor, outputs=output_tensor)
model.summary()
```

### 2. 训练图像分类网络

- **步骤**：
  1. **数据准备**：获取并预处理大规模图像数据集，例如 ImageNet。包括数据增强（旋转、翻转、裁剪等）以提高模型的泛化能力。
  2. **模型构建**：构建卷积神经网络（CNN）模型，包括卷积层、池化层、全连接层等。
  3. **编译模型**：选择适当的损失函数（如交叉熵），优化器（如Adam），以及评价指标（如准确率）。
  4. **训练模型**：使用训练数据进行模型训练，调整超参数（如学习率、批量大小）以优化性能。
  5. **模型评估**：在验证集和测试集上评估模型性能，使用混淆矩阵、ROC曲线等进行深入分析。
  6. **模型优化**：根据评估结果，进行模型优化，如调整网络架构、增加正则化等。

### 3. 使用 MobileNet 调整预训练模型

- **MobileNet**：
  - 轻量级卷积神经网络，适用于移动设备和嵌入式设备。
  - 使用深度可分离卷积来减少计算量和参数数量。

- **步骤**：
  1. **加载预训练模型**：从 Keras 应用中加载预训练的 MobileNet 模型，去掉顶层。
  2. **冻结预训练层**：冻结部分或全部预训练层，以保留预训练的特征提取能力。
  3. **添加新的分类层**：在预训练模型的顶部添加新的全连接层，用于特定任务的分类。
  4. **微调模型**：对新的分类层进行训练，同时可以解冻部分卷积层以进行微调，提高模型在新任务上的性能。

- **代码示例**：

```python
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# 加载预训练的 MobileNet 模型，不包含顶层
base_model = MobileNet(weights='imagenet', include_top=False)

# 添加新的全连接层
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # 10 分类任务

# 构建完整模型
model = Model(inputs=base_model.input, outputs=predictions)

# 冻结预训练的卷积层
for layer in base_model.layers:
    layer.trainable = False

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))
```

### 4. 微调分类器提高准确性

- **步骤**：
  1. **解冻部分卷积层**：解冻部分卷积层，使其参与训练，通常选择靠近顶层的几层。
  2. **使用较小的学习率**：在微调阶段使用较小的学习率，以避免破坏预训练模型中已经学到的特征。
  3. **训练模型**：在较小的学习率下继续训练模型，提高在特定任务上的性能。

---

## 深度学习 C4_W4

### 1. 区分人脸识别与验证

- **人脸识别**：
  - 识别输入人脸的身份。
  - 通常需要一个大规模的人脸数据库，将输入人脸与数据库中的人脸进行匹配。
  - 应用场景：考勤系统、监控系统等。

- **人脸验证**：
  - 验证两张人脸是否属于同一个人。
  - 常用于身份验证场景，如门禁系统、支付验证等。
  - 不需要大规模的人脸数据库，只需要比较两张人脸的相似度。

### 2. 实施单次学习解决人脸识别

- **单次学习**：
  - 通过一个或少量样本学习识别新的人脸。
  - 使用神经网络将人脸映射到高维特征空间（编码），然后计算不同人脸编码之间的距离。

- **代码示例**：

```python
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten
import tensorflow.keras.backend as K

def triplet_loss(y_true, y_pred, alpha=0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = K.sum(K.square(anchor - positive), axis=-1)
    neg_dist = K.sum(K.square(anchor - negative), axis=-1)
    return K.mean(K.maximum(pos_dist - neg_dist + alpha, 0))

def create_shared_network(input_shape):
    # 创建共享网络
    input = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu')(input)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

input_shape = (96, 96, 3)
anchor_input = Input(input_shape, name='anchor_input')
positive_input = Input(input_shape, name='positive_input')
negative_input = Input(input_shape, name='negative_input')

shared_network = create_shared_network(input_shape)

encoded_anchor = shared_network(anchor_input)
encoded_positive = shared_network(positive_input)
encoded_negative = shared_network(negative_input)

loss = Lambda(triplet_loss)([encoded_anchor, encoded_positive, encoded_negative])

model = Model(inputs=[anchor_input, positive_input, negative_input], outputs=loss)
model.compile(optimizer='adam', loss=triplet_loss)
model.summary()
```

### 3. 应用三重损失函数

- **三重损失函数**：
  - 用于训练人脸识别模型，确保同一个人的特征向量更接近，不同人的特征向量更远。
  - **公式**：\[ L = \max(||A - P||^2 - ||A - N||^2 + \alpha, 0) \]，其中 \(A\) 是锚点，\(P\) 是正样本，\(N\) 是负样本，\(\alpha\) 是一个超参数，用于控制正负样本之间的距离差异。

### 4. 将人脸识别视为二元分类问题

- **二元分类**：
  - 将人脸识别问题转化为判断两张人脸是否属于同一个人的问题。
  - 使用孪生网络（Siamese Network）进行二分类。

- **孪生网络**：
  - 孪生网络由两个共享相同权重的子网络组成，每个子网络接收一个输入图像，并输出其高维特征表示。
  - 通过计算两个输出特征向量的欧氏距离，判断两张人脸是否属于同一个人。

- **代码示例**：

```python
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten
import tensorflow.keras.backend as K

def contrastive_loss(y_true, y_pred, margin=1):
    y_true = K.cast(y_true, y_pred.dtype)
    positive_distance = K.square(y_pred)
    negative_distance = K.square(K.maximum(margin - y

_pred, 0))
    return K.mean(y_true * positive_distance + (1 - y_true) * negative_distance)

def create_base_network(input_shape):
    input = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu')(input)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

input_shape = (96, 96, 3)
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(lambda x: K.sqrt(K.sum(K.square(x[0] - x[1]), axis=-1, keepdims=True)))([processed_a, processed_b])

model = Model([input_a, input_b], distance)
model.compile(loss=contrastive_loss, optimizer='adam')
model.summary()
```

---

希望这些详细的笔记内容对您有帮助。如果有任何进一步的要求或修改，请告诉我。